In [1]:
import glob, os
import pandas as pd
import numpy as np
import pickle

from sklearn.externals import joblib
import scorecardpy as sc

import warnings
warnings.filterwarnings('ignore')


/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: ignore
  # Remove the CWD from sys.path while we load stuff.


In [2]:
v1 = '기본'

lst1 = ['y',
       '귀점_당월', '상설_당월', '총동의_당월', '사랑온_당월', '통합보장_당월', 
        '조회_직3P', '환산_당월'
      ,'사랑온_직3P', '상설_직3P', '총동의_직3P','통합보장_직3P','조회_당월','귀점_직3P','환산_직3P'
       ,'유지','가동1'
      ]
with open(f'./result2차/bins_adj_{v1}.pickle', 'rb') as f:
    bins_adj1 = pickle.load(f)

with open(f"./result2차/model_{v1}.pkl","rb") as f:
    lr1 = pickle.load(f)

with open(f'./result2차/card_{v1}.pickle', 'rb') as f:
    card1 = pickle.load(f)

In [3]:
yymm = '2310'

data_tot = pd.read_csv(f'./input/DATA_TOT_{yymm}.csv', 
                    encoding = 'euc-kr',
                   converters={'코드':str, '사업부':str})
data = data_tot[:]

# data = pd.read_csv(f'./input/INPUT_2_2306.csv', 
#                     encoding = 'euc-kr',
#                    converters={'코드':str, '사업부':str})

In [4]:
tmp1 = pd.read_excel(f'./result2차/스코어_2306.xlsx', converters={'코드':str})
tmp2 = pd.read_excel(f'./result2차/스코어_이외_2306.xlsx', converters={'코드':str}).dropna(axis=0)

fclst = list(tmp1.코드) + list(tmp2.코드)
notinnewdata = list(filter(lambda x: x not in list(data.코드), fclst))
len(notinnewdata)

1237

In [ ]:
# tmp1 = pd.read_excel(f'./result2차/스코어_2306.xlsx', converters={'코드':str})
# tmp2 = pd.read_excel(f'./result2차/스코어_이외_2306.xlsx', converters={'코드':str}).dropna(axis=0)

# fclst = list(tmp1.코드) + list(tmp2.코드)
# data = data.iloc[list(map(lambda x: x in fclst, data.코드))]
# data.shape

In [5]:
data.columns = list(map(lambda x: x.replace(f'_{yymm}',''), data.columns))

data = data[~data.귀점_당월.isna()].reset_index(drop=True)

data['환산_직3'] = data['환산_직3'].fillna(0)    
lst = ['귀점','조회','상설','총동의','사랑온','통합보장','환산']
for l in lst:
    data[f'{l}_당월'] = list(map(lambda x: int(x.replace(',','')) if type(x)==str else x, data[f'{l}_당월']))
    data[f'{l}_직3'] = list(map(lambda x: int(x.replace(',','')) if type(x)==str else x, data[f'{l}_직3']))
    data[f'{l}_직3P'] = data[f'{l}_당월'] - data[f'{l}_직3']

    
# input1 = data[lst1]
# input2 = data[lst2]

if 'y' not in data.columns :
    data['y'] =0 
data_iv1 = data[lst1]
# data_iv2 = data[lst2]

# input1 = sc.var_filter(data_iv1, y='y')
# input2 = sc.var_filter(data_iv2, y='y')

In [6]:
# sum(input1.환산_당월!=data_iv1.환산_당월)

In [7]:
woe1 = sc.woebin_ply(data_iv1, bins_adj1)
y1 = woe1.loc[:,'y']
X1 = woe1.loc[:,woe1.columns != 'y']

# woe2 = sc.woebin_ply(data_iv1, bins_adj2)
# y2 = woe1.loc[:,'y']
# X2 = woe2.loc[:,woe2.columns != 'y']

[INFO] converting into woe values ...


In [8]:
with open(f'./result2차/input_{v1}.pickle', 'rb') as f:
    c1 = pickle.load(f)
X1 = X1[c1].fillna(0)

# c2 = ['총동의_당월_woe', '조회_직3P_woe', '상설_당월_woe', '통합보장_당월_woe', '사랑온_당월_woe',
#        '귀점_당월_woe', '환산_당월_woe']
# X2 = X2[c2]


In [9]:
ngrp = 5
n = len(data_iv1)//ngrp
rank_lst = []
for i in range(1,ngrp+1):
    rank_lst += [i]*n

rank_lst += [ngrp]*(len(data)-len(rank_lst))

In [10]:
pred1 = lr1.predict_proba(X1)[:,1]
# pred2 = lr2.predict_proba(X2)[:,1]
pred1

array([0.05779858, 0.03032087, 0.69716348, ..., 0.20412744, 0.22482725,
       0.02505151])

In [11]:
score1 = sc.scorecard_ply(data_iv1, card1, print_step=0)
score1['pred1'] = pred1
score1 = score1.sort_values(by ='score', ascending=False).rename(columns={'score':'score1'})
score1['rank1'] = rank_lst
# score2 = sc.scorecard_ply(input2, card2, print_step=0)
# score2['pred2'] = pred2
# score2 = score2.sort_values(by ='score', ascending=False).rename(columns={'score':'score2'})
# score2['rank2'] = rank_lst


In [12]:
result = data.join(score1).sort_values(by = ['score1'], ascending=[False])
# result = data.join(score1).join(score2).sort_values(by = ['score1', 'score2'], ascending=[False, False])
result.head(1)

,사업부,사업부명,지역단,지역단명,지점,지점명,코드,이름,위촉경로,위촉일자,...,조회_직3P,상설_직3P,총동의_직3P,사랑온_직3P,통합보장_직3P,환산_직3P,y,score1,pred1,rank1
5200,00000320,강남경원,189,수원지역단,2119,오산지점,0001899570,고수경,일반,20230104,...,6.5,11.0,-0.3,0.0,-3.7,-77049.0,0,763.0,0.005419,1


---
# 유형분류 추가

In [13]:
point_all = sc.scorecard_ply(data_iv1, card1, print_step=0, only_total_score=False)
data_all = X1.join(point_all,how='left')

In [14]:
t = data_all.iloc[list(result.index)]

cat1 = ['환산_당월_points']
cat2 = ['조회_당월_points','귀점_당월_points','통합보장_당월_points',   '상설_당월_points']
cat3 = ['사랑온_당월_points','총동의_당월_points']

cat_dict = {}
cat_dict['cat2'] = cat2
cat_dict['cat3'] = cat3

print(cat_dict)

all_cols = sum(list(cat_dict.values()),[])
goodpoint_dict = t[ all_cols ].max()
pd.DataFrame(goodpoint_dict).T



df_case = result[['코드','y', 'score1']]
df_case['환산'] = list(t.환산_당월_points > 0) # q = np.mean(t.환산_당월_points )
# q1 = t[cat2].sum(axis=1).quantile(0.75) # np.mean(t[cat2].sum(axis=1))# 
# q2 = t[cat3].sum(axis=1).quantile(0.75) # np.mean(t[cat3].sum(axis=1)) # 
q1=60; q2=40;
df_case['활동1'] = list(t[cat2].sum(axis=1) > q1) # 18  40   60
df_case['활동2'] = list(t[cat3].sum(axis=1) > q2) # 10  31  40

df_case['case'] = list(map(lambda x,y,z: 1 if x&y&z 
            else (2 if x&~y&z 
                 else (3 if x&y&~z 
                      else (4 if ~x&y&z 
                           else (5 if x&~y&~z 
                                else (6 if ~x&y&~z  
                                      else (7 if ~x&~y&z 
                                           else 8 if ~x&~y&~z else '') ))))), df_case.환산, df_case.활동1, df_case.활동2)) 


df_case['비중'] = 1
grp = df_case.groupby(['case','환산', '활동1', '활동2']).agg({'코드':'count','비중':'sum','score1':'mean','y':'mean'}).sort_values(by='score1',ascending=False)
grp['비중'] = grp['비중']/len(result)*100
grp

{'cat2': ['조회_당월_points', '귀점_당월_points', '통합보장_당월_points', '상설_당월_points'], 'cat3': ['사랑온_당월_points', '총동의_당월_points']}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,,,,코드,비중,score1,y
case,환산,활동1,활동2,,,,
1,True,True,True,1641,19.917466,672.765387,0
2,True,False,True,2320,28.158757,621.814655,0
3,True,True,False,210,2.548853,598.490476,0
4,False,True,True,19,0.230611,563.842105,0
5,True,False,False,1902,23.085326,536.865405,0
6,False,True,False,7,0.084962,508.285714,0
7,False,False,True,162,1.966258,500.253086,0
8,False,False,False,1978,24.007768,359.678463,0


In [15]:
sum(grp.코드)

8239

In [16]:
sum(df_case['환산'])

6073

In [17]:
print(sum(df_case['환산']/len(df_case)), sum(df_case['활동1']/len(df_case)), sum(df_case['활동2']/len(df_case)))

0.7371040174779386 0.22781891006189667 0.5027309139458563


In [19]:
result = result.join(df_case[['환산', '활동1', '활동2', 'case']])
result['그룹'] = list(map(lambda x: 1 if x>=618 else
                         (2 if x>=564 else(
                         3 if x>=493 else(
                         4 if x>=367 else 5))), result.score1))
result.groupby('그룹').agg({ 'y':'mean', 'score1':'min'})

In [22]:
result.rename(columns={'score1':'스코어'}).groupby(['case','환산','활동1','활동2']).agg({'코드':'count', '스코어':'mean'}).reset_index()

,case,환산,활동1,활동2,코드,스코어
0,1,True,True,True,1641,672.765387
1,2,True,False,True,2320,621.814655
2,3,True,True,False,210,598.490476
3,4,False,True,True,19,563.842105
4,5,True,False,False,1902,536.865405
5,6,False,True,False,7,508.285714
6,7,False,False,True,162,500.253086
7,8,False,False,False,1978,359.678463


In [23]:
# result_f = result[['코드','score1','pred1','그룹','환산','활동1','활동2','case']]
# result_f = result_f.sort_values(by=['score1','pred1'], ascending=[False, True])

In [24]:
d1 = result.groupby('score1').pred1.mean().to_frame().reset_index()
pred = [d1.pred1[0]]
for i in range(1,len(d1)):
    if pred[-1]<d1.pred1[i]:
        new = pred[-1] + (pred[-1] - pred[-2])/2
        pred+=[new]
        
    else:
        pred += [d1.pred1[i]]
d1['pred'] = pred
for i in range(len(d1)-1):
    if d1.pred[i]<d1.pred[i+1]:
        print(i)
d1 = d1.set_index('score1').drop(columns=['pred1'])

In [25]:
# p = pd.DataFrame(result.groupby('score1').pred1.median())
# p.columns = ['pred']

r1 = result.set_index('score1')
r1 = r1.join(d1).reset_index()

result_f = r1[['코드','score1','pred','그룹','환산','활동1','활동2','case',f'{yymm}차월']]
result_f = result_f.sort_values(by=['score1','pred'], ascending=[False, True])
result_f

,코드,score1,pred,그룹,환산,활동1,활동2,case,2310차월
8238,0001899570,763.0,0.005419,1,True,True,True,1,10
8236,0001941483,759.0,0.005712,1,True,True,True,1,6
8237,0001824407,759.0,0.005712,1,True,True,True,1,20
8235,0001948058,756.0,0.005912,1,True,True,True,1,5
8234,0001911788,755.0,0.006130,1,True,True,True,1,10
...,...,...,...,...,...,...,...,...,...
2,0001943719,268.0,0.839876,5,False,False,False,8,6
3,0001957761,268.0,0.839876,5,False,False,False,8,4
4,0001969925,268.0,0.839876,5,False,False,False,8,4
1,0001919864,264.0,0.846243,5,False,False,False,8,7


In [26]:
result_f.to_excel(f'./result2차/스코어_{yymm}.xlsx', index=False, encoding='utf-8-sig')

---

In [27]:
# result.sort_values(by=['score1','pred'], ascending=[False, True]).to_excel('./result.xlsx', index=False, encoding='utf-8-sig')

In [28]:
point_all = data[['코드']].join(point_all)
point_all.to_excel(f'./result2차/포인트_{yymm}.xlsx', index=False, encoding='utf-8-sig')

---

In [29]:
po = pd.read_csv(f'./input/파일럿대상자분류.csv', 
                    encoding = 'euc-kr',
                   converters={'FC코드':str, '사업부':str})
po = po[po.FC코드!=''].set_index('FC코드')
po

,유형분류,3점미만 선정時선정 이유
FC코드,,
0001831532,육성불가,NaN
0001677720,육성불가,NaN
0001785565,육성불가,NaN
0001784571,육성&선발,NaN
0001769538,육성불가,NaN
...,...,...
0001684279,육성&未선발,NaN
0001731713,육성&未선발,NaN
0001729874,활동우수,NaN


In [30]:
dd = po.join(result.set_index('코드'))
dd = dd[~dd.y.isna()]
dd

,유형분류,3점미만 선정時선정 이유,사업부,사업부명,지역단,지역단명,지점,지점명,이름,위촉경로,...,환산_직3P,y,score1,pred1,rank1,환산,활동1,활동2,case,그룹
FC코드,,,,,,,,,,,,,,,,,,,,,
0001677720,육성불가,NaN,00308000,부산대구,157.0,부산동부지역단,7601.0,연산지점,김수진,경력,...,NaN,0.0,348.0,0.632205,5.0,False,False,False,8.0,5.0
0001784571,육성&선발,NaN,00308000,부산대구,157.0,부산동부지역단,9771.0,우동지점,서순혜,일반,...,319608.0,0.0,673.0,0.018362,1.0,True,True,True,1.0,1.0
0001769538,육성불가,NaN,00308000,부산대구,157.0,부산동부지역단,9777.0,센텀스타지점,경혜란,일반,...,0.0,0.0,334.0,0.678820,5.0,False,False,False,8.0,5.0
0001804554,육성불가,NaN,00308000,부산대구,157.0,부산동부지역단,1037.0,부곡지점,이예선,일반,...,0.0,0.0,305.0,0.757390,5.0,False,False,False,8.0,5.0
0001791041,육성불가,NaN,00308000,부산대구,157.0,부산동부지역단,9956.0,금정지점,차하린,일반,...,0.0,0.0,313.0,0.734893,5.0,False,False,False,8.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0001684279,육성&未선발,NaN,00304000,강북경인,16.0,강북노원지역단,4212.0,세계로지점,양윤주,일반,...,-68456.0,0.0,627.0,0.034320,2.0,True,False,True,2.0,1.0
0001731713,육성&未선발,NaN,00304000,강북경인,16.0,강북노원지역단,6562.0,강북스타지점,엄정란,일반,...,-459150.0,0.0,645.0,0.026865,2.0,True,False,True,2.0,1.0
0001729874,활동우수,NaN,00304000,강북경인,16.0,강북노원지역단,1141.0,화랑대지점,여경희,일반,...,-1167507.0,0.0,631.0,0.033041,2.0,True,True,True,1.0,1.0


In [31]:
result.groupby(['rank1']).agg({'그룹':'count','y':'mean', 'score1':'max','pred1':'max'})

,그룹,y,score1,pred1
rank1,,,,
1,1647,0,763.0,0.023725
2,1647,0,656.0,0.045996
3,1647,0,607.0,0.098859
4,1647,0,548.0,0.420948
5,1651,0,412.0,0.853383


In [32]:
dd.groupby(['rank1','유형분류']).agg({'그룹':'count','y':'mean', 'score1':'max'})

그룹    y  score1
rank1 유형분류                   
1.0   육성&未선발  16  0.0   723.0
      육성&선발   10  0.0   733.0
      활동우수    27  0.0   744.0
2.0   육성&未선발  17  0.0   651.0
      육성&선발   11  0.0   656.0
      활동우수    22  0.0   653.0
3.0   육성&未선발  16  0.0   604.0
      육성&선발   14  0.0   606.0
      활동우수    15  0.0   601.0
4.0   육성&未선발  26  0.0   545.0
      육성&선발   33  0.0   547.0
      육성불가     9  0.0   546.0
      활동우수    16  0.0   547.0
5.0   육성&未선발  28  0.0   408.0
      육성&선발   26  0.0   412.0
      육성불가    31  0.0   375.0
      활동우수     6  0.0   376.0

---
# 대상자 이외

In [33]:
result2 = pd.read_excel(f'./result2차/대상자이외_{yymm}.xlsx',
                   converters={'코드':str, '사업부':str})
result2 = result2.set_index('score1')

FileNotFoundError: [Errno 2] No such file or directory: './result2차/대상자이외_2310.xlsx'

In [34]:
result2 = result2.join(d1).reset_index()

NameError: name 'result2' is not defined

In [35]:
t = result2[:]
df_case2 = result2[['코드','y', 'score1']]
df_case2['환산'] = list(t.환산_당월_points > 0)
q1=60; q2=40;
df_case2['활동1'] = list(t[cat2].sum(axis=1) > q1) # 18  40   60
df_case2['활동2'] = list(t[cat3].sum(axis=1) > q2) # 10  31  40

df_case2['case'] = list(map(lambda x,y,z: 1 if x&y&z 
            else (2 if x&~y&z 
                 else (3 if x&y&~z 
                      else (4 if ~x&y&z 
                           else (5 if x&~y&~z 
                                else (6 if ~x&y&~z  
                                      else (7 if ~x&~y&z 
                                           else 8 if ~x&~y&~z else '') ))))), df_case2.환산, df_case2.활동1, df_case2.활동2)) 


df_case2['비중'] = 1
grp = df_case2.groupby(['case','환산', '활동1', '활동2']).agg({'코드':'count','비중':'sum','score1':'mean','y':'mean'}).sort_values(by='score1',ascending=False)
grp['비중'] = grp['비중']/len(result2)*100
grp

NameError: name 'result2' is not defined

In [36]:
save_data[save_data.score1==255].pred = 0.865

NameError: name 'save_data' is not defined

In [ ]:
save_data = df_case2.drop(columns=['y','비중']).merge(result2[['코드', 'pred']], on='코드', how='left')
save_data

In [ ]:
save_data

In [ ]:
save_data.to_excel(f'./result2차/스코어_이외_{yymm}.xlsx', index=False, encoding='utf-8-sig')